In [4]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

2. Content-based model

In [5]:
netflix = pd.read_csv('data/netflix_titles.zip')
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [6]:
model = TfidfVectorizer(stop_words='english')

In [7]:
netflix['description'] = netflix['description'].fillna('')

In [8]:
feature_matrix = model.fit_transform(netflix['description'])

In [9]:
feature_matrix.shape

(7787, 17905)

In [10]:
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

In [11]:
indices = pd.Series(netflix.index,index=netflix['title']).drop_duplicates()

In [12]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return netflix['title'].iloc[ind_movie]

In [13]:
get_recommendations("Star Trek")

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

In [14]:
get_recommendations("Balto")

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

3. Коллаборативная фильтрация

In [15]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build scikit-surprise


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-surprise (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [115 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-311\surprise
      copying surprise\accuracy.py -> build\lib.win-amd64-cpython-311\surprise
      copying surprise\builtin_datasets.py -> build\lib.win-amd64-cpython-311\surprise
      copying surprise\dataset.py -> build\lib.win-amd64-cpython-311\surprise
      copying surprise\dump.py -> build\lib.win-amd64-cpython-311\surprise
      copying surprise\reader.py -> build\lib.win-amd64-cpython-311\surprise
      copying surprise\trainset.py -> build\lib.win-amd64-cpython-311\surprise
      copying surprise\utils.py -> build\lib.win-amd64-cpython-311\surprise
      copying surprise\__init__.py -> build\lib.win-amd64-cpython-311\surprise
      copying surprise\__main__.py -> build\lib.win-amd64-cpython-311\surpris

In [16]:
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

ModuleNotFoundError: No module named 'surprise'

In [1]:
! pip install lightfm


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "c:\Projects\DS\lessons\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Projects\DS\lessons\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Projects\DS\lessons\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\zhigu\AppData\Local\Temp\pip-build-env-xr6drqis\overlay\Lib\site-packages\setuptools\build_meta.py", line 334, in get_requires

In [17]:
! pip install tensorflow

  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata (4.9 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached h5py-3.12.1-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached ml_dtypes-0.4.1-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.meta

In [18]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model
import pandas as pd

In [19]:
df = pd.read_csv('data/ratings.csv')

In [20]:
from sklearn.model_selection import train_test_split, cross_val_score

In [23]:
train, test = train_test_split(df, train_size=0.8, random_state=42)

In [24]:
train.shape

(785404, 3)

In [25]:
df.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [27]:
n_books = df.book_id.unique()
len(n_books)

10000

In [28]:
n_users = df.user_id.unique()
len(n_users)

53424

In [29]:
n_books = df["book_id"].nunique()
print(n_books)
n_users = df["user_id"].nunique()
print(n_users)

10000
53424


In [30]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

In [31]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [32]:
conc = Concatenate()([book_vec, user_vec])

In [33]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [34]:
model2 = Model([user_input, book_input], out)

In [35]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [36]:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 99s 4ms/step - loss: 0.9973
Epoch 2/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 105s 4ms/step - loss: 0.6809
Epoch 3/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 108s 4ms/step - loss: 0.6459
Epoch 4/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 107s 4ms/step - loss: 0.6203
Epoch 5/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 95s 4ms/step - loss: 0.5967


In [ ]:
model2.evaluate([test.user_id, test.book_id], test.rating)

In [ ]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)